# Get historical data throgh API 
# Make Api call for crypto and stock(S&P500) from Allpaca
# Make API call for comodities from barchart


In [1]:
# Limit the number of securities for each alloocation( depends on the data fed)

# Get combination of alocated assets and weighting 
# Use KMeans algorithm for clustering 
from sklearn.cluster import KMeans
# from yahoofinancials import YahooFinancials
import yfinance as yf
import numpy as np
import requests
import json
import pandas as pd
import datetime as dt

# Data collection and preparation for Stocks, Crypto, and Commodities

In [2]:
# # Using NASDAQ Data Link API to collect commodities data from the OPEC Crude Oil Basket.
# # Collecting 10-year crude oil market data from OPEC.
# OPEC_url = "https://data.nasdaq.com/api/v3/datasets/OPEC/ORB.json?api_key=9MXjsAzYyYicz7jtVQX5&start_date=2018-02-06"
# OPEC_response = requests.get(OPEC_url)
# OPEC_data = OPEC_response.json()

# print(json.dumps(OPEC_data, indent=4, sort_keys=True))

In [3]:
# # Using NASDAQ Data Link API to collect commodities data from the LBMA London Bullion Market Association.
# # Collecting 10-year gold market data from LBMA.
# LBMA_gold_url = "https://data.nasdaq.com/api/v3/datasets/LBMA/GOLD.json?api_key=9MXjsAzYyYicz7jtVQX5&start_date=2013-02-07"
# LBMA_gold_response = requests.get(LBMA_gold_url)
# LBMA_gold_data = LBMA_gold_response.json()

# print(json.dumps(LBMA_gold_data, indent=4, sort_keys=True))

In [4]:
#Ask user how much is the allocation for Equity Allocation
# Check the input 
while True:
    try:
        equity_allocation = float(input('Please enter the amount of capital you wish to invest :'))
        break
    except:
        print('Please enter a numerical value.')

Please enter the amount of capital you wish to invest : 10000


In [ ]:
# Prompt the user how many years of historical data they wish to pull for the analysis
# Check the input 
while True:
    try:
        num_years = int(input('Please enter the number of years (1-10) of historical data that you wish to retrieve for the analysis :'))
        break
    except:
        print('Please enter a numerical value.'
             )

In [13]:
# VARIATION 3
# Define the lists of asset classes and their respective tickers. Set it as variables, so the selection can be changed easier if neded.
# api_pull = {'crypto' : ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD'],
#             'stocks' : ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META','XOM', 'LAC'],
#             'commodities' : ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']}
# api_pull = {'stocks' : ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META','XOM', 'LAC']}

#api_pull = {'crypto' : ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD']}
api_pull = {'commodities' : ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']}
# Store data for each asset class in a dictionary
# Use .items()to iterate over the key-value pairs of the api_pull dictionary
# pull data for the number of years requested by the user
data = {}
for asset_class, tickers in api_pull.items():
    data[asset_class] = {}
    for ticker in tickers:
        data[asset_class][ticker] = yf.Ticker(ticker).history(period=f"{num_years}y")

In [14]:

# Review pulled data:
data 

{'commodities': {'GC=F':                                   Open         High          Low        Close  \
  Date                                                                            
  2018-02-13 00:00:00-05:00  1324.500000  1329.599976  1323.400024  1328.099976   
  2018-02-14 00:00:00-05:00  1332.800049  1355.500000  1324.000000  1355.500000   
  2018-02-15 00:00:00-05:00  1352.000000  1355.900024  1349.699951  1352.099976   
  2018-02-16 00:00:00-05:00  1354.000000  1361.400024  1344.900024  1353.199951   
  2018-02-20 00:00:00-05:00  1348.400024  1350.199951  1328.500000  1328.800049   
  ...                                ...          ...          ...          ...   
  2023-02-07 00:00:00-05:00  1870.000000  1875.699951  1868.000000  1871.699951   
  2023-02-08 00:00:00-05:00  1872.099976  1880.400024  1872.099976  1877.400024   
  2023-02-09 00:00:00-05:00  1875.300049  1884.599976  1859.800049  1866.199951   
  2023-02-10 00:00:00-05:00  1861.599976  1863.500000  1852.4000

In [15]:
# Create a data frame for each asset class with the close column
# Access the 'Close' column of the data for each asset class
# Extract the values of the data dictionary and then access the 'Close' column of each dataframe. 
#df_crypto = pd.DataFrame({ticker: data['crypto'][ticker]['Close'] for ticker in api_pull['crypto']})
#df_stocks = pd.DataFrame({ticker: data['stocks'][ticker]['Close'] for ticker in api_pull['stocks']})

df_commodities = pd.DataFrame({ticker: data['commodities'][ticker]['Close'] for ticker in api_pull['commodities']})
df_commodities

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2018-02-13 00:00:00-05:00,1328.099976,16.507000,59.189999,3.1545,491.500000,1011.75,147.274994,474.50,76.379997,1228.500000
2018-02-14 00:00:00-05:00,1355.500000,16.858000,60.599998,3.2285,500.600006,1017.25,148.175003,469.75,75.599998,1225.000000
2018-02-15 00:00:00-05:00,1352.099976,16.777000,61.340000,3.2390,500.700012,1024.25,149.824997,478.00,75.410004,1195.000000
2018-02-16 00:00:00-05:00,1353.199951,16.695000,61.680000,3.2430,507.799988,1021.50,149.725006,478.50,75.720001,1201.500000
2018-02-20 00:00:00-05:00,1328.800049,16.424999,61.900002,3.1860,516.799988,1026.50,149.800003,471.75,77.510002,1185.500000
...,...,...,...,...,...,...,...,...,...,...
2023-02-07 00:00:00-05:00,1871.699951,22.117001,77.139999,4.0885,469.500000,1515.25,187.199997,885.75,85.629997,1747.500000
2023-02-08 00:00:00-05:00,1877.400024,22.367001,78.470001,4.0435,454.100006,1519.75,186.449997,896.00,85.370003,1790.000000
2023-02-09 00:00:00-05:00,1866.199951,22.097000,78.059998,4.1055,433.600006,1519.25,186.824997,879.00,85.500000,1811.500000


In [16]:
# Replace all the NaN values in a data frame using the fillna() method. Create a copy of the data frames.
#df_crypto = df_crypto.fillna(0).copy()
#df_stocks = df_stocks.fillna(0).copy()
# df_commodities = df_commodities.fillna(0).copy()

In [17]:
# df_crypto.head()

In [18]:
# # Review first and last 10 rows for crypto assets class
# display(df_crypto.head(10))
# df_crypto.tail(10)

In [19]:
# Review first and last 10 rows for stocks assets class
display(df_stocks.head(10))
df_stocks.tail(10)

NameError: name 'df_stocks' is not defined

In [20]:
# # Review first and last 10 rows for commodities assets class
# display(df_commodities.head(10))
# df_commodities.tail(10)

In [21]:
#Run montecarlo model

In [22]:
df_crypto.index = pd.to_datetime(df_crypto.index)
#df_stocks.index = pd.to_datetime(df_stocks.index)
# df_commodities.index = pd.to_datetime(df_commodities.index)

df_crypto.index = df_crypto.index.tz_convert('UTC')
#df_stocks.index = df_stocks.index.tz_convert('UTC')
## df_commodities.index = df_commodities.index.tz_convert('UTC')

NameError: name 'df_crypto' is not defined

In [23]:
portfolio_data = pd.concat([df_commodities], axis=1)
# pd.merge(df1,df2, how='inner')
portfolio_data

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2018-02-13 00:00:00-05:00,1328.099976,16.507000,59.189999,3.1545,491.500000,1011.75,147.274994,474.50,76.379997,1228.500000
2018-02-14 00:00:00-05:00,1355.500000,16.858000,60.599998,3.2285,500.600006,1017.25,148.175003,469.75,75.599998,1225.000000
2018-02-15 00:00:00-05:00,1352.099976,16.777000,61.340000,3.2390,500.700012,1024.25,149.824997,478.00,75.410004,1195.000000
2018-02-16 00:00:00-05:00,1353.199951,16.695000,61.680000,3.2430,507.799988,1021.50,149.725006,478.50,75.720001,1201.500000
2018-02-20 00:00:00-05:00,1328.800049,16.424999,61.900002,3.1860,516.799988,1026.50,149.800003,471.75,77.510002,1185.500000
...,...,...,...,...,...,...,...,...,...,...
2023-02-07 00:00:00-05:00,1871.699951,22.117001,77.139999,4.0885,469.500000,1515.25,187.199997,885.75,85.629997,1747.500000
2023-02-08 00:00:00-05:00,1877.400024,22.367001,78.470001,4.0435,454.100006,1519.75,186.449997,896.00,85.370003,1790.000000
2023-02-09 00:00:00-05:00,1866.199951,22.097000,78.059998,4.1055,433.600006,1519.25,186.824997,879.00,85.500000,1811.500000


In [27]:
portfolio_data = portfolio_data.fillna(method='ffill')

In [ ]:
# portfolio_data.fillna(0).copy()

In [29]:
portfolio_data_daily_ret = portfolio_data.pct_change(1).dropna()
portfolio_data_daily_ret.head()

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2018-02-14 00:00:00-05:00,0.020631,0.021264,0.023822,0.023459,0.018515,0.005436,0.006111,-0.010011,-0.010212,-0.002849
2018-02-15 00:00:00-05:00,-0.002508,-0.004805,0.012211,0.003252,0.000200,0.006881,0.011135,0.017563,-0.002513,-0.024490
2018-02-16 00:00:00-05:00,0.000814,-0.004888,0.005543,0.001235,0.014180,-0.002685,-0.000667,0.001046,0.004111,0.005439
2018-02-20 00:00:00-05:00,-0.018031,-0.016173,0.003567,-0.017576,0.017724,0.004895,0.000501,-0.014107,0.023640,-0.013317
2018-02-21 00:00:00-05:00,0.000903,0.010898,-0.003554,0.008004,0.018576,0.007550,-0.021028,-0.012719,0.018062,0.008857


In [30]:
# ## Prepare the data for Monte Carlo simulation
# returns = portfolio_data.pct_change() #Calculate returns for each asset
# mean_return = returns.mean()
# cov_matrix = returns.cov()

In [31]:
mean_daily_ret = portfolio_data.pct_change(1).mean()
mean_daily_ret

GC=F     0.000317
SI=F     0.000420
CL=F    -0.002145
HG=F     0.000307
LBS=F    0.000572
ZS=F     0.000423
GF=F     0.000263
KE=F     0.000734
CT=F     0.000270
ZR=F    -0.000334
dtype: float64

In [32]:
portfolio_data.pct_change(1).corr()

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
GC=F,1.000000,0.787051,0.042133,0.243181,0.038787,0.139401,0.015161,0.101663,0.083386,-0.006155
SI=F,0.787051,1.000000,0.062708,0.334439,0.062222,0.168357,0.047302,0.128693,0.138933,-0.000869
CL=F,0.042133,0.062708,1.000000,0.128877,0.061773,0.077451,0.034507,0.011115,0.052903,-0.002202
HG=F,0.243181,0.334439,0.128877,1.000000,0.104221,0.249618,0.027928,0.133183,0.282214,0.013342
LBS=F,0.038787,0.062222,0.061773,0.104221,1.000000,0.072318,0.019097,0.024533,0.048861,-0.034631
ZS=F,0.139401,0.168357,0.077451,0.249618,0.072318,1.000000,-0.044808,0.381704,0.159974,0.057653
GF=F,0.015161,0.047302,0.034507,0.027928,0.019097,-0.044808,1.000000,-0.093831,0.033508,-0.036573
KE=F,0.101663,0.128693,0.011115,0.133183,0.024533,0.381704,-0.093831,1.000000,0.192176,0.072280
CT=F,0.083386,0.138933,0.052903,0.282214,0.048861,0.159974,0.033508,0.192176,1.000000,0.048749
ZR=F,-0.006155,-0.000869,-0.002202,0.013342,-0.034631,0.057653,-0.036573,0.072280,0.048749,1.000000


In [33]:
portfolio_data.head()

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2018-02-13 00:00:00-05:00,1328.099976,16.507000,59.189999,3.1545,491.500000,1011.75,147.274994,474.50,76.379997,1228.5
2018-02-14 00:00:00-05:00,1355.500000,16.858000,60.599998,3.2285,500.600006,1017.25,148.175003,469.75,75.599998,1225.0
2018-02-15 00:00:00-05:00,1352.099976,16.777000,61.340000,3.2390,500.700012,1024.25,149.824997,478.00,75.410004,1195.0
2018-02-16 00:00:00-05:00,1353.199951,16.695000,61.680000,3.2430,507.799988,1021.50,149.725006,478.50,75.720001,1201.5
2018-02-20 00:00:00-05:00,1328.800049,16.424999,61.900002,3.1860,516.799988,1026.50,149.800003,471.75,77.510002,1185.5


In [34]:
# log_ret = np.log(portfolio_data/portfolio_data.shift(1)).dropna()
# log_ret.head()

In [35]:
# log_ret.describe().transpose()

In [36]:
# # Compute pairwise covariance of columns
# log_ret.cov()

In [37]:
# log_ret.cov()*252 # multiply by days

In [38]:
# Set seed (optional)
np.random.seed(101)

# Stock Columns
print('portfolio_data')
print(portfolio_data.columns)
print('\n')

# Create Random Weights
print('Creating Random Weights')
weights = np.array(np.random.random(10))
print(weights)
print('\n')

# Rebalance Weights
print('Rebalance to sum to 1.0')
weights = weights / np.sum(weights)
print(weights)
print('\n')
# log_ret.mean()
# Expected Return
print('Expected Portfolio Return')
exp_ret = np.sum(log_ret.mean() * weights) *252
print(exp_ret)
print('\n')

# Expected Variance
print('Expected Volatility')
exp_vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
print(exp_vol)
print('\n')

# Sharpe Ratio
SR = exp_ret/exp_vol
print('Sharpe Ratio')
print(SR)

portfolio_data
Index(['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F',
       'ZR=F'],
      dtype='object')


Creating Random Weights
[0.51639863 0.57066759 0.02847423 0.17152166 0.68527698 0.83389686
 0.30696622 0.89361308 0.72154386 0.18993895]


Rebalance to sum to 1.0
[0.10499539 0.11602948 0.00578945 0.03487419 0.13933214 0.16954988
 0.0624131  0.18169153 0.14670601 0.03861884]


Expected Portfolio Return


NameError: name 'log_ret' is not defined

In [39]:
num_ports = 15000

all_weights = np.zeros((num_ports,len(portfolio_data.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):

    # Create Random Weights
    weights = np.array(np.random.random(10))

    # Rebalance Weights
    weights = weights / np.sum(weights)
    
    # Save Weights
    all_weights[ind,:] = weights

    # Expected Return
    ret_arr[ind] = np.sum((log_ret.mean() * weights) *252)

    # Expected Variance
    vol_arr[ind] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))

    # Sharpe Ratio
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

NameError: name 'log_ret' is not defined

In [40]:
sharpe_arr.max()

0.0

In [41]:
sharpe_arr.argmax()

0

In [42]:
all_weights[874,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [43]:
max_sr_ret = ret_arr[874]
max_sr_vol = vol_arr[874]

In [ ]:
# Prepare the data for Monte Carlo simulation
returns = portfolio_data.pct_change() # Calculate returns for each asset
mean_return = returns.mean()
cov_matrix = returns.cov()

# Set the number of simulations and trading days
num_simulations = 1000
stocks_n_comodities_num_trading_days = 252
#crypto_num_trading_days = 365

# Create an empty array to hold the simulation results
simulation_results = np.zeros((num_simulations, len(portfolio_data.columns)))

# Loop over the number of simulations
for i in range(num_simulations):
#     # Generate random daily returns for each asset
     daily_returns = np.random.multivariate_normal(mean_return, cov_matrix, stocks_n_comodities_num_trading_days)
     # Calculate the cumulative returns over the simulation period
     simulation_returns = (1 + daily_returns).cumprod(axis=0)
     # Calculate the final portfolio value for each simulation
     portfolio_values = simulation_returns[-1] * portfolio_data.iloc[-1]
     # Save the portfolio values for this simulation
     simulation_results[i, :] = portfolio_values

# Calculate the mean and standard deviation of the simulation results
mean_portfolio_values = simulation_results.mean(axis=0)
std_portfolio_values = simulation_results.std(axis=0)

# Print the results
print("Expected portfolio value: $", round(mean_portfolio_values.sum(), 2))

In [ ]:
print((portfolio_data <= 0).sum())


In [ ]:
portfolio_data.index = portfolio_data.index.tz_convert('UTC')



In [ ]:
# portfolio_data = portfolio_data.mask(portfolio_data <= 0, portfolio_data.mean())
portfolio_data = portfolio_data.mask(portfolio_data <= 0, portfolio_data.mean(), axis=0)


In [ ]:
# print(portfolio_data.isna().sum())

In [ ]:
# eigvals, _ = np.linalg.eig(cov_matrix)
# print(eigvals)

In [ ]:
#Ask user how much is the allocation for Equity Allocation
# Check the input 
while True:
    try:
        equity_alloc = int(input('max equity'))
        break
    except:
        print('thats not an integer')


In [ ]:
#Ask user how much is the allocation for Commodity Allocation
# Check the input 
while True:
    try:
        commodity_alloc = int(input('max commodity'))
        break
    except:
        print('thats not an integer')


In [ ]:
#Ask user how much is the allocation for Crypto Allocation
# Check the input 
while True:
    try:
        crypto_alloc = int(input('max crypto'))
        break
    except:
        print('thats not an integer')


In [ ]:
#Data from 3 assets with their weight amounts
#Limit each equity t0 allocate 10 items 
assets = {'equity':range(0,equity_alloc,10),'commodity':range(0,commodity_alloc,10),'crypto':range(0,crypto_alloc,10)}




In [ ]:
#Allowing only 3 cluster of assets 
kmeans = KMeans(n_clusters=3)



In [ ]:
#Fit the model to the asset data 
kmeans.fit(assets)



In [ ]:
# Plot vizualization fort the clusters 

In [ ]:
# Ru